### Importación de datos



In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

tienda = pd.read_csv(url)
tienda2 = pd.read_csv(url2)
tienda3 = pd.read_csv(url3)
tienda4 = pd.read_csv(url4)

tienda.head()

,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon
0,Asistente virtual,Electrónicos,164300.0,6900.0,16/01/2021,Pedro Gomez,Bogotá,4,Tarjeta de crédito,8,4.60971,-74.08175
1,Mesa de comedor,Muebles,192300.0,8400.0,18/05/2022,Beatriz Morales,Medellín,1,Tarjeta de crédito,4,6.25184,-75.56359
2,Juego de mesa,Juguetes,209600.0,15900.0,15/03/2021,Juan Fernandez,Cartagena,1,Tarjeta de crédito,1,10.39972,-75.51444
3,Microondas,Electrodomésticos,757500.0,41000.0,03/05/2022,Juan Fernandez,Cali,4,Nequi,1,3.43722,-76.52250
4,Silla de oficina,Muebles,335200.0,20200.0,07/11/2020,Maria Alfonso,Medellín,5,Nequi,1,6.25184,-75.56359


#1. Análisis de facturación



In [7]:
for df in [tienda, tienda2, tienda3, tienda4]:
    df["Fecha de Compra"] = pd.to_datetime(df["Fecha de Compra"], errors="coerce", dayfirst=True)
    df["año_mes"] = df["Fecha de Compra"].dt.to_period("M")

In [19]:
from numpy.random.mtrand import f
tiendas = {
    "1": tienda,
    "2": tienda2,
    "3": tienda3,
    "4": tienda4
}

opcion = input("Ingrese la tienda que desea consultar")
if opcion not in tiendas:
  print("Opción no válida")
else:
  df = tiendas[opcion]
  print(f"Resultados para la tienda {opcion}")

  filtrar = input("Desea filtrar por año y mes? (s/n)")
  if filtrar == "s":
    df["Fecha de Compra"] = pd.to_datetime(df["Fecha de Compra"])
    año_disponible = sorted(df["Fecha de Compra"].dt.year.unique())
    año_min = año_disponible[0]
    año_max = año_disponible[-1]
    print(f"Años disponibles: {año_min} - {año_max}")

    año = input("Ingrese el año")
    mes = input("Ingrese el mes")

    mes = mes.zfill(2)
    periodo = f"{año}-{mes}"
    periodo = pd.Period(periodo, freq="M")

    df_filtrado = df[df["año_mes"] == periodo]

    if df_filtrado.empty:
      print(f"No se encontraron datos para el periodo {periodo}")
    else:
      print(f"Resultados para el periodo {periodo}")

      unidades = df_filtrado["Producto"].count()
      print(f"Unidades vendidas en {periodo}: {unidades}")

      monto = df_filtrado["Precio"].sum()
      print(f"Monto total vendido en {periodo}: {monto}")

  else:
    unidades_mes = df.groupby("año_mes")["Producto"].count()
    print("Unidades vendidas por mes:")
    print(unidades_mes)

    monto_mes = df.groupby("año_mes")["Precio"].sum()
    print("Monto total vendido por mes:")
    print(monto_mes)



Ingrese la tienda que desea consultar1
Resultados para la tienda 1
Desea filtrar por año y mes? (s/n)s


KeyboardInterrupt: Interrupted by user

# 2. Ventas por categoría

# 3. Calificación promedio de la tienda


# 4. Productos más y menos vendidos

# 5. Envío promedio por tienda